<a href="https://colab.research.google.com/github/5l0wC0d3r/Moviept/blob/main/moviept.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install peft
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes
!pip install transformers
!pip install optimum


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from optimum.gptq import GPTQQuantizer

In [ ]:

Settings.embed_model = HuggingFaceEmbedding(model_name = "BAAI/bge-small-en-v1.5")
Settings.llm = None
Settings.chunk_size = 4096
Settings.chunk_overlap = 64

In [ ]:
documents = SimpleDirectoryReader("/content/drive/MyDrive/MindCase").load_data()

In [ ]:
print(len(documents))

In [ ]:
#store vectors into vector database
index = VectorStoreIndex.from_documents(documents)

In [ ]:
#set number of docs to retrieve
top_k = 3

#configure retriever
retriever = VectorIndexRetriever(index = index, similarity_top_k = top_k)

In [ ]:
#assemble query engine
query_engine = RetrieverQueryEngine(
    retriever = retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)]
)

In [ ]:
#load fine-tuned model from hub
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=False, revision = "main")

In [ ]:
config = PeftConfig.from_pretrained("shawhin/shawgpt-ft")
model = PeftModel.from_pretrained(model, "shawhin/shawgpt-ft")

In [ ]:
from auto_gptq import exllama_set_max_input_length
model = exllama_set_max_input_length(model, max_input_length=2800)

In [ ]:
#load tokenizer
tokenizer =AutoTokenizer.from_pretrained(model_name, use_fast = True)

In [ ]:
from urllib.parse import quote_from_bytes
query = "Explain the theme of the movie?"
response = query_engine.query(query)

#query documents
context = "Context: \n"
for i in range(top_k):
  context = context + response.source_nodes[i].text + "\n\n"

print(context)

In [ ]:
# prompt (with context)
prompt_template_w_context = lambda context, comment: f"""[INST]MOVIEPT, functioning as a virtual screen writer, with an expertise in Feminist Theory and related domains \
will help provide a detailed solution to the user's query with reference to the context provided.

{context}
Give solution the following query. Use the context above.

{comment}
[/INST]
"""

In [ ]:
comment = "Explain the theme of the movie?"
prompt = prompt_template_w_context(context, comment)

print(prompt)

In [ ]:
print(len(prompt))

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt")
print(len(inputs))

In [ ]:
print(len(inputs['input_ids'][0]))

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

In [ ]:
query = '''Who are the characters?'''

response = query_engine.query(query)

#query documents
context = "Context: \n"
for i in range(top_k):
  context = context + response.source_nodes[i].text + "\n\n"

prompt = prompt_template_w_context(context, query)

print(prompt)

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

In [ ]:
def prompter(comment):
  #query documents
  response = query_engine.query(comment)

  context = "Context: \n"

  for i in range(top_k):
    context = context + response.source_nodes[i].text + "\n\n"

  prompt = prompt_template_w_context(context, comment)

  return prompt

In [ ]:
query = "How many male and female characters are in the movie?"
prompt = prompter(query)

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=1024)

print(tokenizer.batch_decode(outputs)[0])

In [ ]:
def moviept(input_query):
  input_prompt = prompter(input_query)

  inputs = tokenizer(input_prompt, return_tensors="pt")

  outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"),
                           attention_mask=inputs["attention_mask"].to("cuda"),
                           pad_token_id=tokenizer.eos_token_id,
                           max_new_tokens=1024)

  result = tokenizer.batch_decode(outputs)[0]

  solution = result.split("[/INST]\n")[1].strip().replace("</s>", "\n~MTHgpt")
  print(input_query + "\n" + solution)

In [ ]:
query = "Does the script pass the Bechdel test?"
moviept(query)

In [ ]:
moviept("Explain the theme of the movie?")

In [ ]:
moviept("What is the role of Deckard in the movie?")

In [ ]:
moviept("Who are the characters?")

In [ ]:
moviept("How many male and female characters are in the movie?")

In [ ]:
moviept("Does the script pass the Bechdel test?")

In [ ]:
moviept("What is the role of Deckard in the movie?")